# Data Preparation - Version 2

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import datetime
from typing import Dict, List

import boto3
import pandas as pd
from dotenv import find_dotenv, load_dotenv

## About

Updated data processing in preparation for `v2` analysis.

## User Inputs

In [3]:
process_raw_files = False
remove_short_tweets = False

path_to_folder = "/datasets/twitter/kinesis-demo/"
years_wanted = [2021] + [2022]

# List of headers for all streamed twitter attributes
headers = [
    "id",
    "geo",
    "coordinates",
    "place",
    "contributors",
    "is_quote_status",
    "quote_count",
    "reply_count",
    "retweet_count",
    "favorite_count",
    "favorited",
    "retweeted",
    "created_at",
    "source",
    "in_reply_to_user_id",
    "in_reply_to_screen_name",
    "source_text",
    "place_id",
    "place_url",
    "place_place_type",
    "place_name",
    "place_full_name",
    "place_country_code",
    "place_country",
    "place_bounding_box_type",
    "place_bounding_box_coordinates",
    "place_attributes",
    "coords_type",
    "coords_lon",
    "coords_lat",
    "geo_type",
    "geo_lon",
    "geo_lat",
    "user_name",
    "user_screen_name",
    "user_followers",
    "user_friends",
    "user_listed",
    "user_favourites",
    "user_statuses",
    "user_protected",
    "user_verified",
    "user_contributors_enabled",
    "user_joined",
    "user_location",
    "retweeted_tweet",
    "tweet_text_urls",
    "tweet_text_hashtags",
    "tweet_text_usernames",
    "num_urls_in_tweet_text",
    "num_users_in_tweet_text",
    "num_hashtags_in_tweet_text",
    "text",
]
string_cols = [
    "favorited",
    "retweeted",
    # 'is_quote_status',  # contains True, False, 0, 1, 2, 3
    "user_protected",
    "user_verified",
    "user_contributors_enabled",
]
text_cols = [
    "user_name",
    "user_screen_name",
    "user_location",
    "place_name",
    "place_full_name",
    "place_country_code",
    "place_country",
    "tweet_text_urls",
    "source_text",
    "text",
]
int_cols = [
    "quote_count",
    "reply_count",
    # 'retweet_count',  # monirity of rows are text
    "favorite_count",
    "user_followers",
    "user_friends",
    "user_listed",
    "user_favourites",
    "user_statuses",
    "num_urls_in_tweet_text",
    "num_users_in_tweet_text",
    "num_hashtags_in_tweet_text",
]
datetime_cols = ["created_at", "user_joined"]

# List of partial strings to use to filter out unwanted tweets
# - tweets containing sensitive tweet texts that were found retrospectively
#   and should be excluded from the CSV files
unwanted_partial_strings_list = [
    # specific to crypto mining
    "crypto",
    "token",
    "koistarter",
    "daostarter",
    "decentralized",
    "services",
    "pancakeswap",
    "eraxnft",
    "browsing",
    "kommunitas",
    "hosting",
    "internet",
    "exipofficial",
    "servers",
    "wallet",
    "liquidity",
    "rewards",
    "floki",
    "10000000000000linkstelegram",
    "dogecoin",
    "czbinance",
    "watch",
    "binance",
    "dogelonmars",
    "cryptocurrency",
    "hbomax",
    "money",
    "danheld",
    "cybersecurity",
    # others
    "prostitution",
    "nairobi",
    "musembe",
    "volcano detected",
    "block-2",
    "mo-greene",
    "running scared2012",
    "running scared 2012",
    "massacres",
    "eric ephriam chavez",
    "drugs",
    "tanzanite",
    "vvsorigin",
    "gemstonecarat",
    "bin laden",
    "saddam",
    "webuye",
    "bungoma",
    "perished",
    "popescu",
    "whore",
    "nasty",
    "ethereum",
    "pay someone",
    "gamejoin",
    "nft",
    "breeding",
    "seungkwan",
    "woozi",
    "hoshi",
    "bitcrush",
    "arcade",
    "homeworkpay",
    "homework",
    "photocards",
    "deta",
    "marketing",
    "dreamcast",
    "sega",
    "xbox",
    "wii",
    "ps4",
    "kasama",
    "nung",
    "lahat",
    "jinsoul",
    "brunisoul",
    "loona",
    "taas",
    "nung",
    "essay",
    # religious
    "scriptures",
    "methusealah",
    "testament",
    "yahweh",
    "god",
    "mullah",
    "allah",
    "clergy",
    "mercy",
    "morality",
    "muslims,",
    "hindus",
    "buddhist",
    "catholics",
    "christians",
    "atheist",
    # inappropriate
    "nazist",
    "antifa",
    "proud boys",
]

raw_data_filepath = "data/raw/raw_data.parquet.gzip"
proc_data_filepath = "data/processed/processed_data"

blanks_threshold = 0.6
nones_threshold = 0.8
object_dtype_mapper = {
    "id": pd.StringDtype(),
    "source": pd.StringDtype(),
    "is_quote_status": pd.BooleanDtype(),
}
empty_array_cols = ["place_bounding_box_coordinates"]
profanities_word_list_url = (
    "https://raw.githubusercontent.com/ben174/profanity/"
    "master/profanity/data/wordlist.txt"
)

In [4]:
load_dotenv(find_dotenv())
aws_region = os.getenv("AWS_REGION", default="us-east-2")
s3_bucket_name = os.getenv("AWS_S3_BUCKET_NAME")
profanities = pd.read_csv(profanities_word_list_url).squeeze().tolist()

## Get AWS Python SDK Objects

In [5]:
s3_client = boto3.client("s3", region_name=aws_region)

## Helper Functions

In [7]:
def get_hourly_folders_per_day(
    s3_bucket_name: str, path_to_folder: str, years_wanted: List[int]
) -> List[str]:
    """Get list of S3 hourly data folders, per day of streamed data."""
    list_of_hourly_dirs = []
    for year in years_wanted:
        monthly_prefixes = s3_client.list_objects_v2(
            Bucket=s3_bucket_name,
            Prefix=f"{path_to_folder[1:]}{year}/",
            Delimiter="/",
        )["CommonPrefixes"]
        # print(monthly_prefixes)

        for monthly_prefix in monthly_prefixes:
            daily_prefixes = s3_client.list_objects_v2(
                Bucket=s3_bucket_name,
                Prefix=monthly_prefix["Prefix"],
                Delimiter="/",
            )["CommonPrefixes"]
            # print(monthly_prefix, daily_prefixes)

            for daily_prefix in daily_prefixes:
                hourly_prefixes = s3_client.list_objects_v2(
                    Bucket=s3_bucket_name,
                    Prefix=daily_prefix["Prefix"],
                    Delimiter="/",
                )["CommonPrefixes"]
                # print(
                #     monthly_prefix,
                #     # daily_prefixes,
                #     hourly_prefixes,
                # )
                list_of_hourly_dirs.append(hourly_prefixes)
    list_of_hourly_dirs_flat = [sl["Prefix"] for l in list_of_hourly_dirs for sl in l]
    print(f"Found {len(list_of_hourly_dirs_flat):,} hourly folders")
    for hourly_dirs in list_of_hourly_dirs_flat:
        print(hourly_dirs)
    return list_of_hourly_dirs_flat


def read_files_per_hour(
    s3_bucket_name: str, flat_list_of_hourly_dirs: List[str]
) -> List[str]:
    """Read individual files in each hourly folder in the S3 bucket."""
    file_contents_all = []
    for q, list_of_hourly_dirs in enumerate(flat_list_of_hourly_dirs, 1):
        objects_hourly_all = s3_client.list_objects_v2(
            Bucket=s3_bucket_name, Prefix=list_of_hourly_dirs
        )
        file_contents_list = []
        for k, file_obj_dict in enumerate(objects_hourly_all["Contents"], 1):
            file_body = s3_client.get_object(
                Bucket=s3_bucket_name, Key=file_obj_dict.get("Key")
            )["Body"].read()
            print(
                f"Dir {q}/{len(flat_list_of_hourly_dirs)} - "
                f"{list_of_hourly_dirs}, reading object "
                f"{k}/{len(objects_hourly_all['Contents'])}"
            )
            file_contents_list.append(file_body)
        print(
            f"Dir {q}/{len(flat_list_of_hourly_dirs)} - {list_of_hourly_dirs} contains "
            f"{len(file_contents_list):,} file objects"
        )
        file_contents_all.append(file_contents_list)
    file_contents_all_flat = [
        file_contents
        for file_contents_list in file_contents_all
        for file_contents in file_contents_list
    ]
    return file_contents_all_flat


def convert_file_contents_to_df(file_contents_all_flat: List) -> pd.DataFrame:
    """."""
    nested_list_of_records = []
    for file_body in file_contents_all_flat:
        list_of_records = file_body.decode("utf-8").split("\n")[:-1]
        nested_list_of_records.append(list_of_records)
    df = pd.DataFrame(
        [record.split("\t")[:-1] for sl in nested_list_of_records for record in sl]
    )
    return df


def add_column_headers(df: pd.DataFrame, headers: List[str]) -> pd.DataFrame:
    """."""
    num_rows = len(df)
    df_mismatched = df[~df.iloc[:, -2:].isna().all(axis=1)]
    mismatched_rows = len(df_mismatched)
    # with pd.option_context("display.max_columns", None):
    #     display(df_mismatched.style.set_caption(f"{mismatched_rows:,} Mismatched rows"))
    df = df.loc[df.iloc[:, -2:].isna().all(axis=1)].drop(columns=[53, 54])
    assert df.shape[1] == len(headers)
    df.columns = headers
    num_new_rows = len(df)
    print(f"Dropped {(num_rows - num_new_rows):,} mismatched rows out of {num_rows:,}")
    return df


def str_to_bool(str_value: str) -> bool:
    """
    Convert a string to a boolean.

    Source
    ------
    https://python.tutorialink.com/how-to-convert-multiple-pandas-columns-from-string-boolean-to-boolean/
    """
    if str_value == "True":
        return True
    elif str_value == "False":
        return False
    else:  # str_value == 'None':
        return None


def convert_string_cols_to_bool(df, string_cols: List[str]) -> pd.DataFrame:
    """."""
    print("Converting string columns to booleans...")
    for c in string_cols:
        df[c] = df[c].apply(str_to_bool).astype(pd.BooleanDtype())
    print("Done.")
    return df


def convert_text_cols_to_str(df: pd.DataFrame, text_cols: List[str]) -> pd.DataFrame:
    """."""
    print("Converting text columns to strings...")
    for c in text_cols:
        df[c] = df[c].astype(pd.StringDtype())
        df.loc[df[c].isin(["None"]), c] = None  # done for user_location
    print("Done.")
    return df


def convert_text_cols_to_int(df: pd.DataFrame, int_cols: List[str]) -> pd.DataFrame:
    """."""
    print("Converting string columns to integers...")
    for c in int_cols:
        df[c] = df[c].astype(pd.Int16Dtype())
    print("Done.")
    return df


def convert_datetime_cols(df: pd.DataFrame, datetime_cols: List[str]) -> pd.DataFrame:
    """."""
    print("Converting datetime columns...")
    for c in datetime_cols:
        df[c] = pd.to_datetime(df[c])
    print("Done.")
    return df


def drop_nans(
    df: pd.DataFrame, columns_to_drop_nans: List[str] = ["text"]
) -> pd.DataFrame:
    """."""
    print("Dropping columns with missing values...")
    df = df.dropna(how="all")
    df = df.dropna(subset=columns_to_drop_nans)
    print("Done.")
    return df


def drop_unwanted_partial_tweets(
    df: pd.DataFrame,
    columns_to_drop_tweets: List[str],
    unwanted_partial_strings_list: List[str],
) -> pd.DataFrame:
    """."""
    print("Removing texts with unwanted substrings...")
    unwanted_partial_strings = "|".join(unwanted_partial_strings_list)
    for c in columns_to_drop_tweets:
        df = df[~df[c].str.lower().str.contains(unwanted_partial_strings)]
    print("Done.")
    return df.copy()


def drop_retweets(
    df: pd.DataFrame,
    retweet_col: str = "retweeted_tweet",
    retweet_value: str = "yes",
) -> pd.DataFrame:
    """."""
    print("Removing re-tweets...")
    df = df[df[retweet_col] != retweet_value]
    print("Done.")
    return df


def drop_empty_tweets(df: pd.DataFrame, tweet_col: str = "text") -> pd.DataFrame:
    """."""
    print("Removing empty (blank) tweets...")
    df = df[df[tweet_col] != ""]
    print("Done.")
    return df


def save_to_parquet(df: pd.DataFrame, filepath: str) -> None:
    """."""
    print("Saving data to a parquet file...")
    df.to_parquet(filepath, index=False)
    print("Done.")


def upload_file_to_s3(
    local_fpath: str,
    s3_fpath: str,
    s3_bucket_name: str,
    aws_region: str = "us-east-2",
) -> None:
    """Upload single file to S3."""
    print(f"Uploading {local_fpath} to {s3_bucket_name}/{s3_fpath}...")
    s3_client = boto3.client("s3", region_name=aws_region)
    s3_client.upload_file(local_fpath, s3_bucket_name, s3_fpath)
    print("Done.")


def remove_html_ampersand(
    df: pd.DataFrame,
    text_col: str = "text",
    ampersand_html_substring: str = "&amp;",
    replacement_text: str = " & ",
) -> pd.DataFrame:
    """."""
    print(f"Removing {ampersand_html_substring} from text column...")
    df[text_col] = df[text_col].str.replace(
        ampersand_html_substring, replacement_text, regex=False
    )
    print("Done.")
    return df


def remove_multiple_whitespaces(
    df: pd.DataFrame, text_col: str = "text", proc_text_col: str = "processed_text"
) -> pd.DataFrame:
    """."""
    print("Removing multiple whitespaces from text column...")
    df["processed_text"] = df[text_col].str.replace("[^\w\s]", "", regex=True)
    print("Done.")
    return df


def remove_leading_trailing_spaces(
    df: pd.DataFrame, text_col: str = "text"
) -> pd.DataFrame:
    """."""
    print("Removing leading and trailing whitespaces from text column...")
    df[text_col] = df[text_col].str.strip()
    print("Done.")
    return df


def change_text_to_lowercase(df: pd.DataFrame, text_col: str = "text") -> pd.DataFrame:
    """."""
    print("Changing text to lowercase...")
    df[text_col] = df[text_col].str.lower()
    print("Done.")
    return df


def remove_special_characters(df: pd.DataFrame, text_col: str = "text") -> pd.DataFrame:
    """."""
    print("Removing special characters from text column...")
    df[text_col] = df[text_col].str.replace(r"[^a-zA-z]", " ", regex=True)
    print("Done.")
    return df


def remove_punctuation(df: pd.DataFrame, text_col: str = "text") -> pd.DataFrame:
    """."""
    print("Removing punctuation from text column...")
    df[text_col] = df[text_col].str.replace(r"\s+", " ", regex=True)
    print("Done.")
    return df


def get_words_from_text(
    df: pd.DataFrame, text_col: str = "text", words_col: str = "words"
) -> pd.DataFrame:
    """."""
    print("Getting words from text...")
    df[words_col] = df[text_col].str.split("\s+")
    print("Done.")
    return df


def filter_tweets_by_num_words(
    df: pd.DataFrame, words_col: str = "words", min_num_words: int = 15
) -> pd.DataFrame:
    """."""
    print("Filtering tweets by number of words...")
    df = df[df[words_col].str.len() > min_num_words]
    print("Done.")
    return df


def get_non_useful_cols(
    df: pd.DataFrame, nones_threshold: float, blanks_threshold: float
) -> List[List[str]]:
    """."""
    print("Getting lists of non-useful columns...")
    includes_nan_cols = []
    includes_blanks_cols = []
    single_value_cols = []
    for c in df.dtypes.to_frame().loc[df.dtypes == "object"].index.tolist():
        if c not in ["words"]:
            # print(c)
            includes_nans = set(df[c].unique().tolist()) > set(["None"])
            includes_blanks = set(df[c].unique().tolist()) > set([""])
            single_value = df[c].nunique() == 1
            if includes_nans:
                includes_nan_cols.append(c)
            if includes_blanks:
                includes_blanks_cols.append(c)
            if single_value:
                single_value_cols.append(c)

    mostly_nan_or_blanks = []
    for c in includes_nan_cols + includes_blanks_cols:
        nan_frac = len(df[df[c] == "None"]) / len(df)
        blank_frac = len(df[df[c] == ""]) / len(df)
        print(
            f"Column = {c}, Missing fraction: {nan_frac:.2f}, "
            f"Blank fraction: {blank_frac:.2f}"
        )
        if nan_frac > nones_threshold or blank_frac > blanks_threshold:
            mostly_nan_or_blanks.append(c)
    print("Done.")
    return [single_value_cols, mostly_nan_or_blanks]


def drop_columns_from_list(
    df: pd.DataFrame, cols_to_drop: List[str], cols_type: str
) -> pd.DataFrame:
    """."""
    print(f"Dropping {cols_type} columns from list...")
    df = df.drop(columns=cols_to_drop)
    print("Done.")
    return df


def change_object_cols_dtype(
    df: pd.DataFrame, object_cols_to_bool_dict: Dict, mapper_dict: Dict
) -> pd.DataFrame:
    """."""
    print("Changing datatype of object columns...")
    df = df.astype(object_cols_to_bool_dict).astype(mapper_dict)
    print("Done.")
    return df

## Get Raw Data

### Get List of Hourly File Contents

In [ ]:
%%time
if process_raw_files:
    list_of_hourly_dirs_flat = get_hourly_folders_per_day(
        s3_bucket_name, path_to_folder, years_wanted
    )
    file_contents_all_flat = read_files_per_hour(s3_bucket_name, list_of_hourly_dirs_flat)

### Convert List to `DataFrame` and Save to Local Disk

In [ ]:
%%time
if process_raw_files:
    df = (
        convert_file_contents_to_df(file_contents_all_flat)
        .pipe(add_column_headers, headers=headers)
        .pipe(save_to_parquet, filepath=raw_data_filepath)
    )

## (Optional) Load Raw Data

### Read Data from Local Disk

In [ ]:
%%time
if not process_raw_files:
    df = pd.read_parquet(raw_data_filepath).copy()

### Inspect Column Values

Show values for columns of text to be converted to `boolean`s

In [ ]:
for c in string_cols:
    display(df[c].value_counts(dropna=False).to_frame())

Show values for columns of text to be converted to `string`s

In [ ]:
for c in text_cols:
    display(df[c].value_counts(dropna=False).to_frame())
    print(len(df), len(df[df[c].isin(["None"])]), df[c].isna().sum())

Show values for columns of text to be converted to `integer`s

In [ ]:
for c in int_cols:
    display(df[c].value_counts(dropna=False).to_frame())

## Process Data

### Set Data Types for `DataFrame` Columns

In [ ]:
%%time
df = (
    df
    .pipe(convert_string_cols_to_bool, string_cols=string_cols)
    .pipe(convert_text_cols_to_str, text_cols=text_cols)
    .pipe(convert_text_cols_to_int, int_cols=int_cols)
    # .pipe(convert_datetime_cols, datetime_cols=datetime_cols)
)

In [ ]:
print(f"Number of rows after loading data = {len(df):,}")
with pd.option_context("display.max_columns", None):
    display(df.sample(5))

Show values for columns of text converted to `boolean`s

In [ ]:
for c in string_cols:
    display(df[c].value_counts(dropna=False).to_frame())

Show values for columns of text converted to `string`s

In [ ]:
for c in text_cols:
    display(df[c].value_counts(dropna=False).to_frame())
    print(len(df), len(df[df[c].isin(["None"])]), df[c].isna().sum())

Show values for columns of text converted to `integer`s

In [ ]:
for c in int_cols:
    display(df[c].value_counts(dropna=False).to_frame())

## Process Text Column

### Cleaning Text Data

In [ ]:
%%time
df = (
    df
    .pipe(
        remove_html_ampersand,
        text_col="text",
        ampersand_html_substring='&amp;',
        replacement_text=' and ',
    )
    .pipe(change_text_to_lowercase, text_col='text')
    .pipe(
        drop_unwanted_partial_tweets,
        columns_to_drop_tweets=['text'],
        unwanted_partial_strings_list=list(set(unwanted_partial_strings_list + profanities)),
    )
    .pipe(remove_multiple_whitespaces, text_col='text', proc_text_col='processed_text')
    .pipe(remove_leading_trailing_spaces, text_col='processed_text')
    .pipe(remove_special_characters, text_col='processed_text')
    .pipe(remove_punctuation, text_col='processed_text')
    .pipe(get_words_from_text, text_col='processed_text', words_col='words')
)
print(f"Number of rows in processed data = {len(df):,}")

In [ ]:
%%time
assert df["text"].str.contains("&amp;").sum() == 0
assert df["text"].str.contains(
    "|".join(list(set(unwanted_partial_strings_list + profanities)))
).sum() == 0

### Filtering Data Based on Text

Drop rows with missing values

In [ ]:
%%time
df = df.pipe(drop_nans, columns_to_drop_nans=["processed_text"])
print(
    f"Number of rows after dropping missing values in the tweet text column = {len(df):,}"
)

Drop retweets

In [ ]:
%%time
df = df.pipe(drop_retweets, retweet_col="retweeted_tweet", retweet_value="yes")
print(f"Number of rows after removing re-tweets = {len(df):,}")

Drop blank tweets

In [ ]:
%%time
df = df.pipe(drop_empty_tweets, tweet_col="processed_text")
print(f"Number of rows after removing blank tweets = {len(df):,}")

(Optional) Remove short tweets

In [ ]:
%%time
if remove_short_tweets:
    df = df.pipe(filter_tweets_by_num_words, words_col='words', min_num_words=15)
    print(
        "Number of rows in processed data, after filtering out tweets based on "
        f"length of text = {len(df):,}"
    )

## Process Non-Text Columns

### Get Columns Without Useful Information

We will get the following types of columns
- column only contains the text `None` or a blank string as the unique value
- column contains the text `None` or a blank string as one of the unique values
- column contains a single unique value

In [ ]:
%%time
only_nan_cols = []
includes_nan_cols = []
only_blanks_cols = []
includes_blanks_cols = []
single_value_cols = []
for c in df.dtypes.to_frame().loc[df.dtypes == "object"].index.tolist():
    if c not in ["words"]:
        # print(c)
        only_nans = df[c].unique().tolist() == ["None"]
        only_blanks = df[c].unique().tolist() == [""]
        includes_nans = set(df[c].unique().tolist()) > set(["None"])
        includes_blanks = set(df[c].unique().tolist()) > set([""])
        single_value = df[c].nunique() == 1
        if only_nans:
            # print(c, includes_nans, only_nans)
            only_nan_cols.append(c)
        if includes_nans:
            includes_nan_cols.append(c)
        if only_blanks:
            only_blanks_cols.append(c)
        if includes_blanks:
            includes_blanks_cols.append(c)
        if single_value:
            single_value_cols.append(c)
print(only_nan_cols)
print(includes_nan_cols)
print(only_blanks_cols)
print(includes_blanks_cols)
print(single_value_cols)

Drop columns containing a single unique value (by definition, this will drop columns containing only `None` or blank strings)

In [ ]:
%%time
df = df.drop(columns=single_value_cols)
# # not necessary
# df = df.drop(columns=only_nan_cols).drop(columns=only_blanks_cols)

Show columns that include `None` or a blank string

In [ ]:
%%time
# blanks_threshold = 0.6
# nones_threshold = 0.8
mostly_nan_or_blanks = []
for c in includes_nan_cols + includes_blanks_cols:
    print(c, len(df[df[c] == 'None']) / len(df), len(df[df[c] == '']) / len(df))
    if (
        (len(df[df[c] == 'None']) / len(df)) > nones_threshold
        or (len(df[df[c] == '']) / len(df)) > blanks_threshold
    ):
        mostly_nan_or_blanks.append(c)
print(mostly_nan_or_blanks)

Drop columns that include `None` or a blank string above the acceptable threshold

In [ ]:
%%time
df = df.drop(columns=mostly_nan_or_blanks)

Show the datatype and unique values in the remaining `object` columns

In [ ]:
%%time
for c in [
    c
    for c in df.dtypes.to_frame().loc[df.dtypes == "object"].index.tolist()
    if c != "words"
]:
    print(c, df[c].dtype)
    display(df[c].value_counts(normalize=True, dropna=False).to_frame())

Drop columns if they mostly contain an empty `list`

In [ ]:
%%time
# empty_array_cols = ['place_bounding_box_coordinates']
df = df.drop(columns=empty_array_cols)

Change the datatype of the remaining `object` columns

In [ ]:
%%time
object_dtype_mapper = {
#     "id": pd.StringDtype(),
#     'source': pd.StringDtype(),
#     'is_quote_status': pd.BooleanDtype(),
# }
df = df.astype({"is_quote_status": bool}).astype(object_dtype_mapper)

### Show Missing Values per Column

In [ ]:
%%time
(
    df.isna().sum().rename("missing").to_frame().merge(
        df.dtypes.rename("dtype").to_frame(),
        left_index=True,
        right_index=True,
    )
    .assign(num_rows=len(df))
    .assign(frac_missing=lambda df: (df['missing'] / df['num_rows'])*100)
)

## Exploratory Data Analysis

Show tweets with a url, hashtag or username in the text

In [ ]:
%%time
df_uhu = df.loc[
    (df["tweet_text_urls"] != "")
    | (df["tweet_text_hashtags"] != "")
    | (df["tweet_text_usernames"] != ""),
    [
        "text",
        "tweet_text_urls",
        "tweet_text_hashtags",
        "tweet_text_usernames",
    ],
]
with pd.option_context("display.max_colwidth", None):
    display(df_uhu.sample(10))

## Complete Data Processing Pipeline

In [8]:
%%time
if process_raw_files:
    list_of_hourly_dirs_flat = get_hourly_folders_per_day(
        s3_bucket_name, path_to_folder, years_wanted
    )
    file_contents_all_flat = read_files_per_hour(s3_bucket_name, list_of_hourly_dirs_flat)
    df = (
        convert_file_contents_to_df(file_contents_all_flat)
        .pipe(add_column_headers, headers=headers)
        .pipe(save_to_parquet, filepath=raw_data_filepath)
    )
else:
    df = pd.read_parquet(raw_data_filepath).copy()

df = (
    df
    .pipe(convert_string_cols_to_bool, string_cols=string_cols)
    .pipe(convert_text_cols_to_str, text_cols=text_cols)
    .pipe(convert_text_cols_to_int, int_cols=int_cols)
    .pipe(convert_datetime_cols, datetime_cols=datetime_cols)
    .pipe(
        remove_html_ampersand,
        text_col="text",
        ampersand_html_substring='&amp;',
        replacement_text=' and ',
    )
    .pipe(change_text_to_lowercase, text_col='text')
    .pipe(
        drop_unwanted_partial_tweets,
        columns_to_drop_tweets=['text'],
        unwanted_partial_strings_list=list(set(unwanted_partial_strings_list + profanities)),
    )
    .pipe(remove_multiple_whitespaces, text_col='text', proc_text_col='processed_text')
    .pipe(remove_leading_trailing_spaces, text_col='processed_text')
    .pipe(remove_special_characters, text_col='processed_text')
    .pipe(remove_punctuation, text_col='processed_text')
    .pipe(get_words_from_text, text_col='processed_text', words_col='words')
    .pipe(drop_nans, columns_to_drop_nans=["processed_text"])
    .pipe(drop_retweets, retweet_col="retweeted_tweet", retweet_value="yes")
    .pipe(drop_empty_tweets, tweet_col="processed_text")
)

print("Performing sanity checks on modified text column...")
assert df["text"].str.contains("&amp;").sum() == 0
assert df["text"].str.contains(
    "|".join(list(set(unwanted_partial_strings_list + profanities)))
).sum() == 0
print("Done")

single_value_cols, mostly_nan_or_blanks = get_non_useful_cols(
    df, nones_threshold, blanks_threshold
)
df = (
    df
    .pipe(
        drop_columns_from_list,
        cols_to_drop=single_value_cols,
        cols_type="single-value",
    )
    .pipe(
        drop_columns_from_list,
        cols_to_drop=mostly_nan_or_blanks,
        cols_type="mostly None or mostly blank"
    )
    .pipe(
        drop_columns_from_list,
        cols_to_drop=empty_array_cols,
        cols_type="empty array",
    )
    .pipe(
        change_object_cols_dtype,
        object_cols_to_bool_dict={"is_quote_status": bool},
        mapper_dict=object_dtype_mapper,
    )
)

if remove_short_tweets:
    df = df.pipe(filter_tweets_by_num_words, words_col='words', min_num_words=15)

now_dtime = datetime.now().replace(microsecond=0)
filepath_with_ext = (
    f"{proc_data_filepath}__{now_dtime.strftime('%Y%m%d_%H%M%S')}"
    ".parquet.gzip"
)
s3_filepath_with_ext = (
    f"{os.path.basename(proc_data_filepath)}/processed_data__"
    f"{now_dtime.strftime('%Y%m%d_%H%M%S')}.parquet.gzip"
)
df.pipe(save_to_parquet, filepath=filepath_with_ext)
upload_file_to_s3(
    local_fpath=filepath_with_ext,
    s3_fpath=s3_filepath_with_ext,
    s3_bucket_name=s3_bucket_name,
    aws_region=aws_region,
)

Converting string columns to booleans...
Done.
Converting text columns to strings...
Done.
Converting string columns to integers...
Done.
Converting datetime columns...
Done.
Removing &amp; from text column...
Done.
Changing text to lowercase...
Done.
Removing texts with unwanted substrings...
Done.
Removing multiple whitespaces from text column...
Done.
Removing leading and trailing whitespaces from text column...
Done.
Removing special characters from text column...
Done.
Removing punctuation from text column...
Done.
Getting words from text...
Done.
Dropping columns with missing values...
Done.
Removing re-tweets...
Done.
Removing empty (blank) tweets...
Done.
Performing sanity checks on modified text column...
Done
Getting lists of non-useful columns...
Column = place, Missing fraction: 0.99, Blank fraction: 0.00
Column = in_reply_to_user_id, Missing fraction: 0.86, Blank fraction: 0.00
Column = in_reply_to_screen_name, Missing fraction: 0.86, Blank fraction: 0.00
Column = place_id

## Links

1. [Meaning of `&amp;` in the text of a tweet](https://www.hammockforums.net/forum/showthread.php/105012-What-does-amp-amp-mean-see-it-a-lot-and-its-not-in-the-acronym-page)